# Notebook principal para o pacote `robot_dynamics`

Este notebook age como um `main` executável no Colab (ou localmente), clonando o repositório antes de importar o pacote.
- Ajuste `REPO_URL` para o repositório Git que deseja usar.
- Após o clone, o caminho do repositório é adicionado ao `sys.path`, permitindo edições diretas nos módulos.
- Inclui um exemplo mínimo de cinemática e dinâmica para um robô planar de 2 DOF.


In [ ]:
import os
import subprocess
import sys

# Substitua pelo seu repositório (por exemplo, GitHub/Drive).
REPO_URL = "https://github.com/<usuario>/PFPython.git"
TARGET_DIR = "PFPython"

if not os.path.exists(TARGET_DIR):
    subprocess.run(["git", "clone", REPO_URL, TARGET_DIR], check=True)

repo_path = os.path.abspath(TARGET_DIR)
if repo_path not in sys.path:
    sys.path.insert(0, repo_path)

print(f"Usando repositório em: {repo_path}")


## Importar utilitários do pacote
Caso você edite o código em `robot_dynamics/`, basta reexecutar as células abaixo para refletir as mudanças.


In [ ]:
import sympy as sp

from robot_dynamics import (
    RobotModel,
    build_links_from_data,
    dynamics,
    forward_kinematics,
    parse_axis_order,
    spatial_jacobians,
)


## Exemplo rápido (2 DOF planar)
Os símbolos e parâmetros podem ser ajustados conforme necessário.
A gravidade foi configurada ao longo do eixo +Y para igualar as convenções de sinais do exemplo planar do livro do Niku.


In [ ]:
from IPython.display import display, Markdown

# Símbolos
q1, q2 = sp.symbols("q1 q2")
dq1, dq2 = sp.symbols("dq1 dq2")
ddq1, ddq2 = sp.symbols("ddq1 ddq2")
l1, l2 = sp.symbols("l1 l2")
m1, m2 = sp.symbols("m1 m2")
g = sp.symbols("g")

# Dados DH (theta, d, a, alpha) e eixos
# Aqui os comprimentos dh (a1, a2) são iguais aos comprimentos das barras (l1, l2)
dh_params = [
    (q1, 0, l1, 0),
    (q2, 0, l2, 0),
]
joint_types, axes = parse_axis_order(["z", "z"])
masses = [m1, m2]
centers_of_mass = [
    sp.Matrix([l1 / 2, 0, 0]),
    sp.Matrix([l2 / 2, 0, 0]),
]
inertias = [
    sp.diag(0, 0, m1 * l1**2 / 12),
    sp.diag(0, 0, m2 * l2**2 / 12),
]

links = build_links_from_data(
    [q1, q2],
    joint_types,
    axes,
    dh_params,
    masses,
    centers_of_mass,
    inertias,
)
robot = RobotModel(links=links, gravity=sp.Matrix([0, g, 0]))

Ts, origins = forward_kinematics(robot)
Jvs, Jws = spatial_jacobians(robot, Ts, origins)

replacements, kinetic_opt, M_opt, C_opt, H_opt, G_opt, tau_opt = dynamics(
    robot,
    [q1, q2],
    [dq1, dq2],
    [ddq1, ddq2],
    parallel=False,
)

sub_map = dict(replacements)

def fully_simplify(expr, passes=2):
    result = sp.Matrix(expr) if hasattr(expr, "shape") else expr

    def _simplify_once(e):
        return sp.simplify(sp.trigsimp(sp.together(sp.expand(e))))

    for _ in range(passes):
        if hasattr(result, "applyfunc"):
            result = result.applyfunc(_simplify_once)
        else:
            result = _simplify_once(result)
    return result


def apply_replacements(expr):
    result = sp.Matrix(expr) if hasattr(expr, "shape") else expr
    symbols_to_replace = set(sub_map.keys())
    while result.atoms(sp.Symbol) & symbols_to_replace:
        result = result.xreplace(sub_map)
    return fully_simplify(result)

replacement_lines = [
    f"- ${sp.latex(sym)} = {sp.latex(expr)}$" for sym, expr in replacements
]

display(Markdown("### Subexpressões extraídas (x_i)"))
if replacement_lines:
    display(Markdown("
".join(replacement_lines)))
else:
    display(Markdown("- Nenhuma subexpressão extraída."))

T_ee = Ts[-1]
dq_vec = sp.Matrix([dq1, dq2])
ddq_vec = sp.Matrix([ddq1, ddq2])

# Versões expandidas (sem x_i) para comparação
kinetic_expanded = apply_replacements(kinetic_opt)
M_expanded = apply_replacements(M_opt)
C_expanded = apply_replacements(C_opt)
H_expanded = apply_replacements(H_opt)
G_expanded = apply_replacements(G_opt)
tau_expanded = apply_replacements(tau_opt)
C_vector = tau_expanded - M_expanded * ddq_vec - G_expanded

# Equações de referência (Niku) para duas barras delgadas (I = m*l^2/12)
c1, c2, c12 = sp.cos(q1), sp.cos(q2), sp.cos(q1 + q2)
s2 = sp.sin(q2)
M_niku = sp.Matrix(
    [
        [
            sp.Rational(1, 3) * m1 * l1**2
            + m2 * l1**2
            + sp.Rational(1, 3) * m2 * l2**2
            + m2 * l1 * l2 * c2,
            sp.Rational(1, 2) * m2 * l2**2 + sp.Rational(1, 2) * m2 * l1 * l2 * c2,
        ],
        [
            sp.Rational(1, 2) * m2 * l2**2 + sp.Rational(1, 2) * m2 * l1 * l2 * c2,
            sp.Rational(1, 3) * m2 * l2**2,
        ],
    ]
)
coriolis_niku = sp.Matrix(
    [
        -sp.Rational(1, 2) * m2 * l1 * l2 * s2 * (2 * dq1 * dq2 + dq2**2),
        sp.Rational(1, 2) * m2 * l1 * l2 * s2 * dq1**2,
    ]
)
gravity_niku = sp.Matrix(
    [
        (sp.Rational(1, 2) * m1 + m2) * g * l1 * c1 + sp.Rational(1, 2) * m2 * g * l2 * c12,
        sp.Rational(1, 2) * m2 * g * l2 * c12,
    ]
)
tau_niku = M_niku * ddq_vec + coriolis_niku + gravity_niku

sections = [
    ("Cinemática direta (T_ee)", T_ee),
    ("Jacobiano linear Jv (elo final)", Jvs[-1]),
    ("Jacobiano angular Jw (elo final)", Jws[-1]),
    ("Energia cinética total", kinetic_expanded),
    ("Matriz de inércia M", M_expanded),
    ("Matriz de Coriolis/Centrífuga C", C_expanded),
    ("Matriz H = C + G", H_expanded),
    ("Vetor de gravidade G", G_expanded),
    ("Vetor de Coriolis/Centrífuga (C*dq)", C_vector),
    ("Torques τ", tau_expanded),
]

for title, value in sections:
    display(Markdown(f"### {title}"))
    display(value)

comparisons = [
    ("M de Niku", M_niku),
    ("Diferença M (nosso - Niku)", fully_simplify(M_expanded - M_niku)),
    ("Vetor C de Niku (C*dq)", coriolis_niku),
    (
        "Diferença vetor C (nosso - Niku)",
        fully_simplify(C_vector - coriolis_niku),
    ),
    ("Vetor G de Niku", gravity_niku),
    ("Diferença G (nosso - Niku)", fully_simplify(G_expanded - gravity_niku)),
    ("τ de Niku", tau_niku),
    ("Diferença τ (nosso - Niku)", fully_simplify(tau_expanded - tau_niku)),
]

for title, value in comparisons:
    display(Markdown(f"### {title}"))
    display(value)
